In [ ]:
!nvidia-smi

Tue Aug 23 10:57:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd /content/drive/MyDrive/Datasets

/content/drive/MyDrive/Datasets


In [ ]:
!ls

tomato.zip


In [ ]:
!apt-get install -y zip unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
zip is already the newest version (3.0-11build1).
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
%cd ~/

/root


In [ ]:
!mkdir WORKSPACE

In [ ]:
!cp /content/drive/MyDrive/Datasets/tomato.zip WORKSPACE/

In [ ]:
%cd WORKSPACE
!ls

/root/WORKSPACE
tomato.zip


In [ ]:
!unzip tomato.zip

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/image-seg/image-background-remove-tool

/content/drive/MyDrive/Colab Notebooks/image-seg/image-background-remove-tool


In [ ]:
!ls

carvekit			     docker-compose.cuda.yml  results
conftest.py			     Dockerfile.cpu	      results_crp
crop_image			     Dockerfile.cuda	      results_neo
demo_crop_image.py		     docs		      results_neo_2
demo_multi_process.py		     leaf		      results_neo_3
demo_multi_thread.py		     LICENSE		      results_old
demo_multi_thread_tuple_limitles.py  my_leaf.jpg	      setup.py
demo_multi_thread_tuple.py	     README.md		      tests
demo.py				     requirements_test.txt
docker-compose.cpu.yml		     requirements.txt


### **Image Segmentation**

In [ ]:
import os, time, torch
from multiprocessing.pool import ThreadPool
from carvekit.api.high import HiInterface

In [ ]:
srcPath = "/root/WORKSPACE/tomato/combined_train"
DEST_PATH = "/root/WORKSPACE/tomamto_op/combined_train"

In [ ]:
paths = os.listdir(srcPath)  # returns list
full_paths = list(map(lambda path: os.path.join("./", srcPath, path), paths))
filter_path = [i for i in full_paths if i.endswith('.jpg') or i.endswith('.png') or i.endswith('jpeg')]

# filter_path = filter_path[0: 10]  # select first 10 item

In [ ]:
os.makedirs(DEST_PATH, exist_ok=True)

In [ ]:
interface = HiInterface(batch_size_seg=5, batch_size_matting=1,
                        device='cuda' if torch.cuda.is_available() else 'cpu',
                        seg_mask_size=320, matting_mask_size=1024)


In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")

GPU available: True


In [ ]:
from PIL import ImageFilter
from PIL import Image

def find_top_pixel(image: Image):
    for y in range(image.height):
        for x in range(image.width):
            r, g, b, a = image.getpixel((x, y))
            # print(f"x: {x:3}, y: {y:3} -> {(r, g, b, a)}")
            if a == 255:
                return y
    return -1


def find_bottom_pixel(image: Image):
    for y in range(image.height - 1, -1, -1):
        for x in range(image.width):
            r, g, b, a = image.getpixel((x, y))
            # print(f"x: {x:3}, y: {y:3} -> {(r, g, b, a)}")
            if a == 255:
                return y
    return -1


def find_left_pixel(image: Image):
    for x in range(image.width):
        for y in range(image.height):
            r, g, b, a = image.getpixel((x, y))
            # print(f"x: {x:3}, y: {y:3} -> {(r, g, b, a)}")
            if a == 255:
                return x
    return -1


def find_right_pixel(image: Image):
    for x in range(image.width - 1, -1, -1):
        for y in range(image.height):
            r, g, b, a = image.getpixel((x, y))
            # print(f"x: {x:3}, y: {y:3} -> {(r, g, b, a)}")
            if a == 255:
                return x
    return -1


def find_crop_edges(image: Image):
    left = find_left_pixel(image)
    top = find_top_pixel(image)
    right = find_right_pixel(image)
    bottom = find_bottom_pixel(image)
    return left, top, right, bottom


def crop_object_and_strech(image: Image) -> Image:
    og_size = image.size
    filtered_img = image.filter(ImageFilter.MedianFilter)
    crop_edge = find_crop_edges(filtered_img)
    return image.crop(crop_edge).resize(og_size)


In [ ]:
def get_name_path(img_path: str) -> str:
    return img_path.split('/')[-1].split(".")[0]


def image_process(img_path):
    start_t = time.perf_counter()
    img_name = get_name_path(img_path)
    print(f"\n'now working on image at {img_name}'")
    image = interface([img_path])[0]
    image = crop_object_and_strech(image)
    background = Image.new('RGBA', image.size, (128, 128, 128))
    image = Image.alpha_composite(background, image)
    f_name = f"{img_name}_crp_strc.jpg"
    dest = os.path.join(DEST_PATH, f_name)
    image.convert("RGB").save(dest, 'JPEG', quality=80)
    end_t = time.perf_counter()
    return f_name, end_t - start_t

In [ ]:
start_g_t = time.perf_counter()
with ThreadPool() as pool:
    results = pool.imap_unordered(image_process, filter_path)
    for f_name, duration in results:
        print(f"{f_name} completed in {duration:.2f} seconds")
end_g_t = time.perf_counter()
g_duration = end_g_t - start_g_t
print(f"process done in {g_duration:.2f} seconds")


'now working on image at AppleCedarRust1'

'now working on image at TomatoHealthy4'

'now working on image at PotatoHealthy1'AppleCedarRust1_crp_strc.jpg completed in 0.70 seconds


'now working on image at TomatoEarlyBlight2'TomatoHealthy4_crp_strc.jpg completed in 0.70 seconds


'now working on image at PotatoEarlyBlight1'PotatoHealthy1_crp_strc.jpg completed in 0.57 seconds


'now working on image at TomatoEarlyBlight4'
TomatoEarlyBlight2_crp_strc.jpg completed in 0.61 seconds

'now working on image at PotatoEarlyBlight3'TomatoEarlyBlight4_crp_strc.jpg completed in 0.59 seconds


'now working on image at PotatoEarlyBlight5'PotatoEarlyBlight1_crp_strc.jpg completed in 0.63 seconds


'now working on image at AppleCedarRust4'PotatoEarlyBlight3_crp_strc.jpg completed in 0.54 seconds


'now working on image at PotatoEarlyBlight2'PotatoEarlyBlight5_crp_strc.jpg completed in 0.57 seconds


'now working on image at TomatoHealthy1'AppleCedarRust4_crp_strc.jpg completed in 0.53 seconds


'no

In [ ]:
!zip -r test.zip /root/WORKSPACE/tomamto_op/test

  adding: root/WORKSPACE/tomamto_op/test/ (stored 0%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoYellowCurlVirus3_crp_strc.jpg (deflated 6%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoHealthy4_crp_strc.jpg (deflated 2%)
  adding: root/WORKSPACE/tomamto_op/test/PotatoEarlyBlight1_crp_strc.jpg (deflated 1%)
  adding: root/WORKSPACE/tomamto_op/test/PotatoEarlyBlight4_crp_strc.jpg (deflated 2%)
  adding: root/WORKSPACE/tomamto_op/test/AppleScab3_crp_strc.jpg (deflated 1%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoHealthy2_crp_strc.jpg (deflated 3%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoYellowCurlVirus2_crp_strc.jpg (deflated 3%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoEarlyBlight3_crp_strc.jpg (deflated 3%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoHealthy3_crp_strc.jpg (deflated 1%)
  adding: root/WORKSPACE/tomamto_op/test/TomatoYellowCurlVirus4_crp_strc.jpg (deflated 2%)
  adding: root/WORKSPACE/tomamto_op/test/AppleScab2_crp_strc.jpg (deflated 3%)
  add